In [1]:
RDATE = 20220930
NYEAR =365

import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan

LN = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\01_Loans\\"+str(RDATE)+"\infocube_01_loan"+str(RDATE)+".txt", sep = ",", header = 0, low_memory = False)
LN.columns = LN.columns.str.upper()
LN.M_ACCOUNT_NO = LN.M_ACCOUNT_NO.astype(str)

OD  = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\02_OD\\"+str(RDATE)+"\\infocube_02_OD"+str(RDATE)+".txt", sep = ",", header = 0, low_memory = False)
OD.columns = OD.columns.str.upper()
OD.ACCOUNT_NO = OD.ACCOUNT_NO.astype(str)

INV  = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\03_Investment\\"+str(RDATE)+"\infocube_03_inv"+str(RDATE)+".txt", sep = ",", header = 0, low_memory = False)
INV.columns = INV.columns.str.upper()
INV.M_ACCOUNT_NO = INV.M_ACCOUNT_NO.astype(str)

TRADE  = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\04_Trade\\"+str(RDATE)+"\infocube_04_Trade"+str(RDATE)+".txt", sep = ",", header = 0, low_memory = False)
TRADE.columns = TRADE.columns.str.upper()
TRADE.ACCOUNT_NO = TRADE.ACCOUNT_NO.astype(str)

RDMS = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\04_MIB_Rdms\\"+str(RDATE)+"\RDMS_"+str(RDATE)+".txt", sep = ",", header = 0, error_bad_lines = False)
RDMS.columns = RDMS.columns.str.upper()
RDMS.ACCOUNT_NO = RDMS.ACCOUNT_NO.astype(str)

TOTAL_IMPAIRED_FINAL  = pd.read_csv(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\6.0 Impaired Loan\\'+str(RDATE)[:4]+'\\'+str(RDATE)[:6]+'\working\Total_Impaired_Final_'+str(RDATE)+'.txt',sep = ",", header = 0, low_memory = False)
TOTAL_IMPAIRED_FINAL.columns = TOTAL_IMPAIRED_FINAL.columns.str.upper()
TOTAL_IMPAIRED_FINAL.V_ORIGINAL_ACCOUNT_NUMBER = TOTAL_IMPAIRED_FINAL.V_ORIGINAL_ACCOUNT_NUMBER.astype(str)

#pakai yg lme sbb xde yg baru (202108) row 450
RSME = pd.read_excel(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\03_Retail_SME_Financing\05_Dataset\202108\RSME_202108.xlsx", sheet_name='RSME')
RSME.columns = RSME.columns.str.strip()
RSME.columns = RSME.columns.str.upper()


LN.columns = LN.columns.str.strip()
LN.columns = LN.columns.str.upper()

LN.M_SUB_MARKET_SEGMENT_CD = LN.M_SUB_MARKET_SEGMENT_CD.astype(float)
LN.M_SUB_MARKET_SEGMENT_CD.fillna(0, inplace=True)

LN.M_FULL_WRITEOFF_FLG = LN.M_FULL_WRITEOFF_FLG.astype(str)
LN.M_FULL_WRITEOFF_FLG = LN.M_FULL_WRITEOFF_FLG.str.strip()
LN.M_FULL_WRITEOFF_FLG = LN.M_FULL_WRITEOFF_FLG.str.upper()
LN.M_FULL_WRITEOFF_FLG.fillna('NA', inplace=True)

LN.M_ACCT_STATUS_CD = LN.M_ACCT_STATUS_CD.astype(float)
LN.M_ACCT_STATUS_CD.fillna(0, inplace=True)

LN.LEVEL_6 = LN.LEVEL_6.astype(str)
LN.LEVEL_6 = LN.LEVEL_6.str.strip()
LN.LEVEL_6 = LN.LEVEL_6.str.upper()
LN.LEVEL_6.fillna('NA', inplace=True)

LN.M_PRODUCT_HIERARCHY_CD = LN.M_PRODUCT_HIERARCHY_CD.astype(str)
LN.M_PRODUCT_HIERARCHY_CD = LN.M_PRODUCT_HIERARCHY_CD.str.strip()
LN.M_PRODUCT_HIERARCHY_CD = LN.M_PRODUCT_HIERARCHY_CD.str.upper()
LN.M_PRODUCT_HIERARCHY_CD.fillna('NA', inplace=True)


LN1 = LN.iloc[np.where((LN.M_SUB_MARKET_SEGMENT_CD==1)&\
                 (LN.M_FULL_WRITEOFF_FLG=='N')&\
                 (LN.M_ACCT_STATUS_CD==1)&\
                 (LN.LEVEL_6!='SALARY')&\
                 (~(LN.M_PRODUCT_HIERARCHY_CD.isin(['711','714'])) )\
                )] 


LN1['CA_ACCOUNT_NO'] = 'NA'
LN1['ID_NUMBER'] = 'NA'
LN1.rename(columns = {'M_APPROVED_LIMIT_SUM':'M_APPROVED_LIMIT',\
                     'M_BNM_BALANCE_SUM':'M_BNM_BALANCE',\
                     'AVG_BOOK_BAL_SUM':'AVG_BOOK_BAL',\
                     'M_DATE_FIRST_RELEASED':'DATE_OPENED',\
                     'M_INTEREST_INCOME_MYR_SUM':'M_INTEREST_INCOME_MYR',\
                     'M_NON_INTEREST_INCOME_SUM':'M_NON_INTEREST_INCOME',\
                     'CUR_GROSS_RATE_SUM':'CUR_GROSS_RATE',\
                     'M_PENALTY_CHARGES_SUM':'M_PENALTY_CHARGES',\
                     'M_MISC_CHARGES_SUM':'M_MISC_CHARGES'}, inplace = True)
LN1['TOTAL_ACTIVITY_COST'] = 0
LN1['MAINTENANCE_COST'] = 0
LN1['FEES_EARNED_MYR'] = 0
LN1['COMMISSION_EARNED_MYR'] = 0
LN1['SOURCE'] = 'LOAN'
LN1['COMMON_COA_ID'] = 0

L001 = LN1[['AS_OF_DATE','M_ACCOUNT_NO','CA_ACCOUNT_NO','ID_NUMBER','M_CUS_NO','M_SOURCE_PROD_TYPE_CD','SOURCE_PROD_TYPE_DSC',\
           'M_PRODUCT_HIERARCHY_CD','LEVEL_6','M_ECONOMIC_SECTOR_CD','ECON_SECTOR','BRANCH_CODE','REGION_DESC',\
           'STATE_DESC','BRANCH_DESC'\
,'M_APPROVED_LIMIT', 'M_BNM_BALANCE','AVG_BOOK_BAL','DATE_OPENED'\
,'M_DATE_AA_APPROVED'\
,'AMRT_TERM'\
,'AMRT_TERM_MULT'\
,'M_MTHS_INSTALLMENT_IN_ARREARS'\
,'M_RESCHEDULE_FLG'\
,'M_RESTRUCTURE_FLG'\
,'M_SPECIAL_MENTION_ACCT_FLG'\
,'M_NPL_FLG'\
,'M_NEW_NPL_FLG'\
,'M_FULL_WRITEOFF_FLG'\
,'M_ACCT_STATUS_CD'\
,'M_INTEREST_INCOME_MYR'\
,'M_FUNDS_CHARGE_MYR'\
,'M_NON_INTEREST_INCOME'\
,'CUR_GROSS_RATE'\
,'M_PENALTY_CHARGES'\
,'M_MISC_CHARGES'\
,'TOTAL_ACTIVITY_COST'\
,'MAINTENANCE_COST'\
,'FEES_EARNED_MYR'\
,'COMMISSION_EARNED_MYR'\
,'M_SUB_MARKET_SEGMENT_CD'\
,'M_SUB_MARKET_SEGMENT_DESC'\
,'EAD_PROPORTIONAL_MYR'\
,'EXPECTED_LOSS_MYR'\
,'RWA_AFTER_RISK_MIT_MYR'\
,'NOB_CD'\
,'M_FINANCING_CONCEPT'\
,'SOURCE'\
,'COMMON_COA_ID']]

OD.columns = OD.columns.str.upper()
OD.columns = OD.columns.str.strip()

OD.M_SUB_MARKET_SEGMENT_CD = OD.M_SUB_MARKET_SEGMENT_CD.astype(float)
OD.M_SUB_MARKET_SEGMENT_CD.fillna(0, inplace=True)

OD.FULL_WRITEOFF_FLG = OD.FULL_WRITEOFF_FLG.astype(str)
OD.FULL_WRITEOFF_FLG = OD.FULL_WRITEOFF_FLG.str.strip()
OD.FULL_WRITEOFF_FLG = OD.FULL_WRITEOFF_FLG.str.upper()
OD.FULL_WRITEOFF_FLG.fillna('NA', inplace=True)

OD.ACCT_STATUS_CD = OD.ACCT_STATUS_CD.astype(float)
OD.ACCT_STATUS_CD.fillna(0, inplace=True)

OD.LEVEL_6 = OD.LEVEL_6.astype(str)
OD.LEVEL_6 = OD.LEVEL_6.str.strip()
OD.LEVEL_6 = OD.LEVEL_6.str.upper()
OD.LEVEL_6.fillna('NA', inplace=True)

OD1 = OD.iloc[np.where((OD.M_SUB_MARKET_SEGMENT_CD==1)&\
                 (OD.FULL_WRITEOFF_FLG=='N')&\
                 (OD.ACCT_STATUS_CD!=3)&\
                 (OD.LEVEL_6!='SMF')\
                )] 

OD1['CA_ACCOUNT_NO'] = 'NA'
OD1['ID_NUMBER'] = 'NA'

OD1.loc[OD1.COMMON_COA_ID==12000,'M_BNM_BALANCE'] = OD1.M_BNM_BALANCE
OD1.loc[~(OD1.COMMON_COA_ID==12000),'M_BNM_BALANCE'] = 0

OD1.loc[OD1.COMMON_COA_ID==12000,'AVG_BOOK_BAL'] = OD1.AVG_BOOK_BAL
OD1.loc[~(OD1.COMMON_COA_ID==12000),'AVG_BOOK_BAL'] = 0

OD1.rename(columns = {'ACCOUNT_OPEN_DATE':'DATE_OPENED'}, inplace = True)

OD1['M_PENALTY_CHARGES_SUM'] = 0
OD1['M_MISC_CHARGES_SUM'] = 0
OD1['FEES_EARNED_MYR'] = 0
OD1['COMMISSION_EARNED_MYR'] = 0
OD1['SOURCE'] = 'OD'

D001 = OD1[['AS_OF_DATE'\
,'ACCOUNT_NO'\
,'CA_ACCOUNT_NO'\
,'ID_NUMBER'\
,'CUS_NO'\
,'SOURCE_PROD_TYPE_CD'\
,'SOURCE_PROD_TYPE_DSC'\
,'PRODUCT_HIERARCHY_CD'\
,'LEVEL_6'\
,'ECONOMIC_SECTOR_CD'\
,'ECON_SECTOR'\
,'BRANCH_CODE'\
,'REGION_DESC'\
,'STATE_DESC'\
,'BRANCH_DESC'\
,'AUTHORISED_LIMIT'\
,'M_BNM_BALANCE'\
,'AVG_BOOK_BAL'\
,'DATE_OPENED'\
,'DATE_AA_APPROVED'\
,'AMRT_TERM'\
,'AMRT_TERM_MULT'\
,'M_MTHS_IN_ARREARS'\
,'RESCHEDULE_FLG'\
,'RESTRUCTURE_FLG'\
,'SPECIAL_MENTION_ACCT_FLG'\
,'NPL_FLG'\
,'NEW_NPL_FLG'\
,'FULL_WRITEOFF_FLG'\
,'ACCT_STATUS_CD'\
,'INTEREST_INCOME_MYR'\
,'FUNDS_CHARGE_MYR'\
,'NON_INTEREST_INCOME'\
,'CUR_GROSS_RATE'\
,'M_PENALTY_CHARGES_SUM'\
,'M_MISC_CHARGES_SUM'\
,'TOTAL_ACTIVITY_COST'\
,'MAINTENANCE_COST'\
,'FEES_EARNED_MYR'\
,'COMMISSION_EARNED_MYR'\
,'M_SUB_MARKET_SEGMENT_CD'\
,'M_SUB_MARKET_SEGMENT_DESC'\
,'EAD_PROPORTIONAL_MYR'\
,'EXPECTED_LOSS_MYR'\
,'RWA_AFTER_RISK_MIT_MYR'\
,'NOB_CD'\
,'M_FINANCING_CONCEPT'\
,'SOURCE'\
,'COMMON_COA_ID']]

INV.columns = INV.columns.str.upper()
INV.columns = INV.columns.str.strip()

INV.M_SUB_MARKET_SEGMENT_CD = INV.M_SUB_MARKET_SEGMENT_CD.astype(float)
INV.M_SUB_MARKET_SEGMENT_CD.fillna(0, inplace=True)

INV.LEVEL_6 = INV.LEVEL_6.astype(str)
INV.LEVEL_6 = INV.LEVEL_6.str.strip()
INV.LEVEL_6 = INV.LEVEL_6.str.upper()
INV.LEVEL_6.fillna('NA', inplace=True)

INV1=INV.iloc[np.where((INV.M_SUB_MARKET_SEGMENT_CD==1)&(INV.LEVEL_6.isin(['SHORT TERM REVOLVING CREDIT','FOREIGN CURRENCY LOAN','OWNED BA DISCOUNTED'])))]

INV1['CA_ACCOUNT_NO'] = 'NA'
INV1['M_APPROVED_LIMIT'] = 0

INV1['DATE_OPENED'] = INV1.ISSUE_DATE
INV1['M_MTHS_IN_ARREARS'] = 0
INV1['RESCHEDULE_FLG'] = 'NA'
INV1['RESTRUCTURE_FLG'] = 'NA'
INV1['SPECIAL_MENTION_ACCT_FLG'] = 'NA'
INV1['NEW_NPL_FLG'] = 'NA'
INV1['FULL_WRITEOFF_FLG'] = 'NA'
INV1['ACCT_STATUS_CD'] = 0
INV1['M_PENALTY_CHARGES_SUM'] = 0
INV1['M_MISC_CHARGES_SUM'] = 0
INV1['TOTAL_ACTIVITY_COST'] = 0
INV1['FEES_EARNED_MYR'] = 0
INV1['COMMISSION_EARNED_MYR'] = 0
INV1['SOURCE'] = 'INV'
INV1['M_FINANCING_CONCEPT'] = 0

I001 = INV1[['AS_OF_DATE'\
,'M_ACCOUNT_NO'\
,'CA_ACCOUNT_NO'\
,'ID_NUMBER'\
,'M_CUS_NO'\
,'M_SOURCE_PROD_TYPE_CD'\
,'SOURCE_PROD_TYPE_DSC'\
,'M_PRODUCT_HIERARCHY_CD'\
,'LEVEL_6'\
,'M_ECONOMIC_SECTOR_CD'\
,'ECON_SECTOR'\
,'BRANCH_CODE'\
,'REGION_DESC'\
,'STATE_DESC'\
,'BRANCH_DESC'
,'M_APPROVED_LIMIT'\
,'M_BNM_BALANCE'\
,'M_AVG_BOOK_BAL_MYR'\
,'DATE_OPENED'\
,'ISSUE_DATE'\
,'AMRT_TERM'\
,'AMRT_TERM_MULT'\
,'M_MTHS_IN_ARREARS'\
,'RESCHEDULE_FLG'\
,'RESTRUCTURE_FLG'\
,'SPECIAL_MENTION_ACCT_FLG'\
,'M_NPL_FLG'\
,'NEW_NPL_FLG'\
,'FULL_WRITEOFF_FLG'\
,'ACCT_STATUS_CD'\
,'M_INTEREST_INCOME_MYR'\
,'M_FUNDS_CHARGE_MYR'\
,'M_NON_INTEREST_INCOME_MYR'\
,'CUR_GROSS_RATE'\
,'M_PENALTY_CHARGES_SUM'\
,'M_MISC_CHARGES_SUM'\
,'TOTAL_ACTIVITY_COST'\
,'M_MAINTENANCE_COST'\
,'FEES_EARNED_MYR'\
,'COMMISSION_EARNED_MYR'\
,'M_SUB_MARKET_SEGMENT_CD'\
,'M_SUB_MARKET_SEGMENT_DESC'\
,'EAD_PROPORTIONAL_MYR'\
,'EXPECTED_LOSS_MYR'\
,'RWA_AFTER_RISK_MIT_MYR'\
,'NOB_CD'\
,'M_FINANCING_CONCEPT'\
,'SOURCE'\
,'COMMON_COA_ID']]


TRADE.columns = TRADE.columns.str.upper()
TRADE.columns = TRADE.columns.str.strip()

TRADE.M_SUB_MARKET_SEGMENT_CD = TRADE.M_SUB_MARKET_SEGMENT_CD.astype(float)
TRADE.M_SUB_MARKET_SEGMENT_CD.fillna(0, inplace=True)

TRADE.PDT_DESCRIPTION = TRADE.PDT_DESCRIPTION.astype(str)
TRADE.PDT_DESCRIPTION = TRADE.PDT_DESCRIPTION.str.strip()
TRADE.PDT_DESCRIPTION = TRADE.PDT_DESCRIPTION.str.upper()
TRADE.PDT_DESCRIPTION.fillna('NA', inplace=True)

TRADE.FUNDED_FLG = TRADE.FUNDED_FLG.astype(str)
TRADE.FUNDED_FLG = TRADE.FUNDED_FLG.str.strip()
TRADE.FUNDED_FLG = TRADE.FUNDED_FLG.str.upper()
TRADE.FUNDED_FLG.fillna('NA', inplace=True)

TRADE1=TRADE.iloc[np.where((TRADE.M_SUB_MARKET_SEGMENT_CD==1)&~(TRADE.PDT_DESCRIPTION.isin(['OWN IAB ACCEPT DISCOUNT', 'ISLAMIC - ONSHORE FOREIGN CURRENCY LOAN', 'ISLAMIC ONSHORE FOREIGN CURRENCY FINANCING']))&(TRADE.FUNDED_FLG=='Y'))]

TRADE1['ID_NUMBER'] = 'NA'
TRADE1['SOURCE'] = 'TRADE'
TRADE1['M_APPROVED_LIMIT'] = 0
TRADE1['DATE_OPENED'] = TRADE1.ISSUE_DATE
TRADE1['M_MTHS_IN_ARREARS'] = 0
TRADE1['RESCHEDULE_FLG'] = 'NA'
TRADE1['RESTRUCTURE_FLG'] = 'NA'
TRADE1['SPECIAL_MENTION_ACCT_FLG'] = 'NA'
TRADE1['NPL_FLG'] = 'NA'
TRADE1['NEW_NPL_FLG'] = 'NA'
TRADE1['FULL_WRITEOFF_FLG'] = 'NA'
TRADE1['ACCT_STATUS_CD'] = 0
TRADE1['M_PENALTY_CHARGES_SUM'] = 0
TRADE1['M_MISC_CHARGES_SUM'] = 0

T001 = TRADE1[['AS_OF_DATE'\
,'ACCOUNT_NO'\
,'CA_ACCOUNT_NO'\
,'ID_NUMBER'\
,'CUS_NO'\
,'SOURCE_PROD_TYPE_CD'\
,'SOURCE_PROD_TYPE_DSC'\
,'PRODUCT_HIERARCHY_CD'\
,'PDT_DESCRIPTION'\
,'ECONOMIC_SECTOR_CD'\
,'ECON_SECTOR'\
,'BRANCH_CODE'\
,'REGION_DESC'\
,'STATE_DESC'\
,'BRANCH_DESC'\
,'M_APPROVED_LIMIT'\
,'M_BNM_BALANCE'\
,'AVG_BOOK_BAL_MYR'\
,'DATE_OPENED'\
,'ISSUE_DATE'\
,'AMRT_TERM'\
,'AMRT_TERM_MULT'\
,'M_MTHS_IN_ARREARS'\
,'RESCHEDULE_FLG'\
,'RESTRUCTURE_FLG'\
,'SPECIAL_MENTION_ACCT_FLG'\
,'NPL_FLG'\
,'NEW_NPL_FLG'\
,'FULL_WRITEOFF_FLG'\
,'ACCT_STATUS_CD'\
,'INTEREST_INCOME_MYR'\
,'FUNDS_CHARGE_MYR'\
,'NON_INTEREST_INCOME_MYR'\
,'CUR_GROSS_RATE'\
,'M_PENALTY_CHARGES_SUM'\
,'M_MISC_CHARGES_SUM'\
,'TOTAL_ACTIVITY_COST'\
,'MAINTENANCE_COST'\
,'FEES_EARNED_MYR'\
,'COMMISSION_EARNED_MYR'\
,'M_SUB_MARKET_SEGMENT_CD'\
,'M_SUB_MARKET_SEGMENT_DESC'\
,'EAD_PROPORTIONAL_MYR'\
,'EXPECTED_LOSS_MYR'\
,'RWA_AFTER_RISK_MIT_MYR'\
,'NOB_CD'\
,'M_FINANCING_CONCEPT'\
,'SOURCE'\
,'COMMON_COA_ID']]



#/*RESULT*/
L001.columns = D001.columns = I001.columns = T001.columns 
frames = [ L001, D001, I001, T001] 
A001 = pd.concat(frames, ignore_index=True)

RDMS1 = RDMS.iloc[np.where(~(RDMS.RSME_RATING_GRADE.isin([0,'0']))& ~(RDMS.RSME_RATING_GRADE.isnull()) )][['CUS_NO','RSME_RATING_GRADE']]

A002 = A001.merge(RDMS1[['CUS_NO','RSME_RATING_GRADE']], on='CUS_NO', how='left').drop_duplicates()

A002.RSME_RATING_GRADE.fillna('UNRATED',inplace=True)

A002.EXPECTED_LOSS_MYR.fillna(0,inplace=True)
A002.RWA_AFTER_RISK_MIT_MYR.fillna(0,inplace=True)
A002.EAD_PROPORTIONAL_MYR.fillna(0,inplace=True)

A002['RD'] = A002.EXPECTED_LOSS_MYR/A002.EAD_PROPORTIONAL_MYR
A002['CRWA'] = A002.RWA_AFTER_RISK_MIT_MYR/A002.EAD_PROPORTIONAL_MYR

A002.loc[A002.RSME_RATING_GRADE.isin(["R1 - C","R1 - D","R1 - E"]),'RG'] = "01.R1"
A002.loc[A002.RSME_RATING_GRADE.isin(["R2 - A","R2 - B"]) ,'RG'] =  "02.R2"
A002.loc[A002.RSME_RATING_GRADE.isin(["R3"]) ,'RG'] =  "03.R3"
A002.loc[A002.RSME_RATING_GRADE.isin(["R4"]) ,'RG'] =  "04.R4"
A002.loc[A002.RSME_RATING_GRADE.isin(["R5"]) ,'RG'] =  "05.R5"
A002.loc[A002.RSME_RATING_GRADE.isin(["R6"]) ,'RG'] =  "06.R6"
A002.loc[A002.RSME_RATING_GRADE.isin(["R7"]) ,'RG'] =  "07.R7"
A002.loc[A002.RSME_RATING_GRADE.isin(["R8"]) ,'RG'] =  "08.R8"
A002.loc[A002.RSME_RATING_GRADE.isin(["R9"]) ,'RG'] =  "09.R9"
A002.loc[A002.RSME_RATING_GRADE.isin(["R10"]) ,'RG'] =  "10.R10"
A002.loc[A002.RSME_RATING_GRADE.isin(["R11"]) ,'RG'] =   "11.R11"
A002.loc[A002.RG.isna(),'RG'] = "20.UNRATED"


#/****************** Added in to get Pricing by Risk Rating on 26 March 2019***********************/

A002.loc[A002.RSME_RATING_GRADE.isin(["R1 - C","R1 - D","R1 - E","R2 - A","R2 - B"]),'RG_BY_PROFITRATE'] = "01.R1 - R2"
A002.loc[A002.RSME_RATING_GRADE.isin(["R3","R4"]),'RG_BY_PROFITRATE'] = "02.R3 - R4"
A002.loc[A002.RSME_RATING_GRADE.isin(["R5","R6","R7","R8"]),'RG_BY_PROFITRATE'] = "03.R5 - R8"
A002.loc[A002.RSME_RATING_GRADE.isin(["R9","R10","R11"]),'RG_BY_PROFITRATE'] = "04.R9 - R11"
A002.loc[A002.RG_BY_PROFITRATE.isna(),'RG_BY_PROFITRATE'] = "20.UNRATED"


A002.loc[A002.CUR_GROSS_RATE.isna(), 'PROFIT_RATE'] = "06.NA"
A002.loc[A002.CUR_GROSS_RATE<0, 'PROFIT_RATE'] = "06.NA"
A002.loc[(A002.CUR_GROSS_RATE>= 0) & (A002.CUR_GROSS_RATE<4), 'PROFIT_RATE'] = "01.0% - 4%"
A002.loc[(A002.CUR_GROSS_RATE>= 4) & (A002.CUR_GROSS_RATE<6), 'PROFIT_RATE'] = "02.4% - 6%"
A002.loc[(A002.CUR_GROSS_RATE>= 6) & (A002.CUR_GROSS_RATE<8), 'PROFIT_RATE'] = "03.6% - 8%"
A002.loc[(A002.CUR_GROSS_RATE>= 8) & (A002.CUR_GROSS_RATE<10),'PROFIT_RATE'] = "04.8% - 10%"
A002.loc[A002.CUR_GROSS_RATE>=10, 'PROFIT_RATE'] = "05.>10%"


A002['ANN_INTEREST_INCOME'] = (A002.INTEREST_INCOME_MYR	 / int(str(RDATE)[-2:])) * NYEAR 
A002['NET_INTEREST_INCOME']= A002.INTEREST_INCOME_MYR	-A002.FUNDS_CHARGE_MYR
A002['ANN_NET_INTEREST_INCOME'] = (A002.NET_INTEREST_INCOME / int(str(RDATE)[-2:]) )* NYEAR 


#/****************** Added in to get impaired flag from Amirul's file (since July 17)***********************/
IMPAIRED = TOTAL_IMPAIRED_FINAL.drop_duplicates()
IMPAIRED['IMPAIRED_FLAG'] = 'Y'

IMPAIRED.loc[IMPAIRED.F_EXPOSURE_DEFAULT_STATUS_FLAG=='Y','NPF_TAG']='NPF'
IMPAIRED.loc[IMPAIRED.F_EXPOSURE_DEFAULT_STATUS_FLAG=='N','NPF_TAG']='PF'

IMPAIRED['F_IMPAIRED_FLAG']=IMPAIRED.IMPAIRED_FLAG

#combine
RSME_BASE00 = A002.merge(IMPAIRED[['V_ORIGINAL_ACCOUNT_NUMBER','OUTSTANDING_AFTER_EIR_UWI','IMPAIRED_FLAG','NPF_TAG','F_IMPAIRED_FLAG','PROD_LV1','F_NCIF_CUST','F_NCIF_ACCT']].rename(columns={'V_ORIGINAL_ACCOUNT_NUMBER':'ACCOUNT_NO'}), on='ACCOUNT_NO', how='left').merge(RSME[['M_ACCOUNT_NO','SRC_PROD_TYP_CD']].rename(columns ={'M_ACCOUNT_NO':'ACCOUNT_NO'}),on='ACCOUNT_NO',how='left')

RSME_BASE00['NPF_TAG'].fillna('NA', inplace=True)

RSME_BASE00.F_IMPAIRED_FLAG.fillna('N',inplace=True)

#/*create summary table for Yield and NPM, and also exposure by risk grading*/
RSME_BASE00.RG.fillna('NA', inplace=True)
RSME_BASE00.M_BNM_BALANCE.fillna(0, inplace=True)
RSME_BASE00.AVG_BOOK_BAL_MYR.fillna(0, inplace=True)
RSME_BASE00.ANN_INTEREST_INCOME.fillna(0, inplace=True)
RSME_BASE00.ANN_NET_INTEREST_INCOME.fillna(0, inplace=True)

RSME_BASE00['NOA'] = 1

SUMM1_YIELDNPM = RSME_BASE00.groupby(['RG'])[['M_BNM_BALANCE','NOA','AVG_BOOK_BAL_MYR','ANN_INTEREST_INCOME','ANN_NET_INTEREST_INCOME']].sum().reset_index()

SUMM1_YIELDNPM.rename(columns = {'M_BNM_BALANCE':'M_BNM_BALANCE_SUM',\
                                                'AVG_BOOK_BAL_MYR':'AVG_BOOK_BAL_SUM',\
                                                'ANN_INTEREST_INCOME':'ANN_INT_INCOME_SUM',\
                                                'ANN_NET_INTEREST_INCOME':'ANN_NET_INT_INCOME_SUM'},inplace=True)

SUMM_YIELDNPM = SUMM1_YIELDNPM
SUMM_YIELDNPM['YIELD'] = SUMM_YIELDNPM.ANN_INT_INCOME_SUM / SUMM_YIELDNPM.AVG_BOOK_BAL_SUM 
SUMM_YIELDNPM['NPM'] = SUMM_YIELDNPM.ANN_NET_INT_INCOME_SUM / SUMM_YIELDNPM.AVG_BOOK_BAL_SUM 

SUMM_YIELDNPM = SUMM_YIELDNPM[['RG','M_BNM_BALANCE_SUM','NOA','YIELD','NPM']]

RSME_BASE00.PROFIT_RATE.fillna('NA', inplace=True)
RSME_BASE00.RG_BY_PROFITRATE.fillna('NA', inplace=True)

SUMM_PROFITRATE = RSME_BASE00.groupby(['RG_BY_PROFITRATE','PROFIT_RATE'])[['M_BNM_BALANCE']].sum().reset_index().rename(columns = {'M_BNM_BALANCE':'M_BNM_BALANCE_SUM'})

RSME_BASE00.EAD_PROPORTIONAL_MYR.fillna(0, inplace=True)
RSME_BASE00.EXPECTED_LOSS_MYR.fillna(0, inplace=True)
RSME_BASE00.RWA_AFTER_RISK_MIT_MYR.fillna(0, inplace=True)

SUMM_RSME = RSME_BASE00

SUMM_RSME['CARW'] = SUMM_RSME.RWA_AFTER_RISK_MIT_MYR / SUMM_RSME.EAD_PROPORTIONAL_MYR 
SUMM_RSME['RD'] = SUMM_RSME.EXPECTED_LOSS_MYR / SUMM_RSME.EAD_PROPORTIONAL_MYR 

SUMM_RSME = SUMM_RSME.groupby(['RG'])[['M_BNM_BALANCE','EAD_PROPORTIONAL_MYR','EXPECTED_LOSS_MYR','RWA_AFTER_RISK_MIT_MYR','CARW','RD']].sum().reset_index()

#/****************** Grouping product codes ***********************/
RSME_BASE = RSME_BASE00

RSME_BASE.loc[RSME_BASE.SOURCE_PROD_TYPE_CD.isin(['S46']), 'M_SOURCE_PROD_TYPE_CD2'] = '02. SME PGX-I -S46'
RSME_BASE.loc[RSME_BASE.SOURCE_PROD_TYPE_CD.isin(['S49','S52']), 'M_SOURCE_PROD_TYPE_CD2'] = '05. RSME/BB portfolio Guarantee-I -S49,S52'
RSME_BASE.loc[RSME_BASE.SOURCE_PROD_TYPE_CD.isin(['H74','H95','W29','W4','W5','T28','T29','T32','W41','W62']), 'M_SOURCE_PROD_TYPE_CD2'] =  '08. SME PBF-i H74,H95,W29,W4,W5,T28,T29,T32,W41,W62'
RSME_BASE.loc[RSME_BASE.SOURCE_PROD_TYPE_CD.isin(['T38']), 'M_SOURCE_PROD_TYPE_CD2'] = '14. SME SMF-I - T38'
RSME_BASE.loc[RSME_BASE.SOURCE_PROD_TYPE_CD.isin(['S17','S18']), 'M_SOURCE_PROD_TYPE_CD2'] = '16. Extension of GTFS-I - S17, S18'
RSME_BASE.loc[RSME_BASE.SOURCE_PROD_TYPE_CD.isin(['H10']), 'M_SOURCE_PROD_TYPE_CD2'] = '18. Business financing for Medical Practitioners -H10'
RSME_BASE.loc[RSME_BASE.SOURCE_PROD_TYPE_CD.isin(['W7']), 'M_SOURCE_PROD_TYPE_CD2'] = '19. SME Clean Financing-I - W7'
RSME_BASE.loc[RSME_BASE.SOURCE_PROD_TYPE_CD.isin(['S11']), 'M_SOURCE_PROD_TYPE_CD2'] = '20. SME Franchise financing-I - S11' 
RSME_BASE.loc[RSME_BASE.SOURCE_PROD_TYPE_CD.isin(['W41','W42']), 'M_SOURCE_PROD_TYPE_CD2'] = '21. SME ECER SME PBF-I W41, W42'
RSME_BASE.loc[RSME_BASE.SOURCE_PROD_TYPE_CD.isin(['S50']), 'M_SOURCE_PROD_TYPE_CD2'] = '22. SME ECER SME PG-I - S50'
RSME_BASE.loc[RSME_BASE.SOURCE_PROD_TYPE_CD.isin(['T34']), 'M_SOURCE_PROD_TYPE_CD2'] = '23. SME SME Micro Financing-I - T34'
RSME_BASE.loc[RSME_BASE.M_SOURCE_PROD_TYPE_CD2.isna(), 'M_SOURCE_PROD_TYPE_CD2'] = '99.OTHERS'


#/********MIA 1,2,3 EXCLUDE IF**********/

RSME_BASE.M_BNM_BALANCE.fillna(0,inplace=True)

RSME_MIA1 = RSME_BASE.iloc[np.where((RSME_BASE.M_MTHS_IN_ARREARS.isin([1,2,3]))&(RSME_BASE.F_IMPAIRED_FLAG=='N'))][['M_MTHS_IN_ARREARS','IMPAIRED_FLAG','M_BNM_BALANCE']]

#RSME_MIA1.M_MTHS_IN_ARREARS = RSME_MIA1.M_MTHS_IN_ARREARS.astype(str)
RSME_MIA1.IMPAIRED_FLAG.fillna('N',inplace=True)

RSME_MIA = RSME_MIA1.groupby(['M_MTHS_IN_ARREARS','IMPAIRED_FLAG'])[['M_BNM_BALANCE']].sum().reset_index()

RSME_MIA['M_BNM_BALANCE_MIL'] = RSME_MIA.M_BNM_BALANCE/1000000

writer = pd.ExcelWriter(r'T:\\MIB Risk Management\\Credit Risk Analytics - Historical\\04_Infocube\\07_RSME\\'+str(RDATE)+'\\RSME'+str(RDATE)+'.xlsx',engine='xlsxwriter')

RSME_BASE.to_excel(writer, sheet_name='RSME', index = False)
SUMM_RSME.to_excel(writer, sheet_name='SUMM_RSME', index = False)
SUMM_YIELDNPM.to_excel(writer,sheet_name='SUMM_YIELDNPM', index = False)
SUMM_PROFITRATE.to_excel(writer, sheet_name='SUMM_PROFITRATE', index = False)
RSME_MIA.to_excel(writer, sheet_name='RSME_MIA', index = False)

writer.save()

KeyboardInterrupt: 